#### Load Necessary Modules

In [ ]:
import json
import os.path as op
import shutil
from pprint import pprint

import mne
from mne.datasets import sample

from mne_bids import (
    BIDSPath,
    make_dataset_description,
    print_dir_tree,
    read_raw_bids,
    write_meg_calibration,
    write_meg_crosstalk,
    write_raw_bids,
)
from mne_bids.stats import count_events

#### Creating Data Path and Event Dictionary

In [ ]:
data_path='/home/rakshita/rds_folders/Sproject1/EmptyRoom/Recordings'
event_dict = {
    "cue/right": 1,
    "cue/left": 2,
    "trialstart":300,
    "stimonset":400,
    "catchonset":500,
    "dot/right": 6,
    "dot/left": 7,
    "resp": 800,
    "blkstart": 900,
    "blkend":1000,
    "expend":1100,
    "cue/noresp":1300
}

raw_fname = op.join(data_path,"Rec8.fif")
er_fname = op.join(data_path,"Rec1.fif")  # empty room
events_fname = op.join(data_path,"Rec8_events.fif")
output_path = op.join(data_path, "MNE-bids_rec11Sep")

#### Delete the output directory if exists

In [ ]:
if op.exists(output_path):
    shutil.rmtree(output_path)

#### Load Raw files from original OPM subject recording and Emptyroom Recording

In [ ]:
raw = mne.io.read_raw(raw_fname)
raw_er = mne.io.read_raw(er_fname)

#### Change All channel types to 'mag' channels except the'stim' channel

In [ ]:
ch_names = raw.info['ch_names']
ch_types = raw.get_channel_types()

channel_types_opm = {}
for ch_name, ch_type in zip(ch_names, ch_types):
    if ch_type != 'stim':
        channel_types_opm[ch_name] = 'mag'

# Set all channel types to 'mag' except 'stim'
raw.set_channel_types(channel_types_opm)
#raw.get_channel_types()

In [ ]:
ch_names_er = raw_er.info['ch_names']
ch_types_er = raw_er.get_channel_types()

channel_types_opm_er = {}
for ch_name_er, ch_type_er in zip(ch_names_er, ch_types_er):
    if ch_type_er != 'stim':
        channel_types_opm_er[ch_name_er] = 'mag'

# Set all channel types to 'mag' except 'stim'
raw_er.set_channel_types(channel_types_opm_er)
#raw_er.get_channel_types()

#### Convert to BIDS format

In [ ]:
# specify power line frequency as required by BIDS
raw.info["line_freq"] = 60
raw_er.info["line_freq"] = 60

subject = '01'  
session = '01'  
task = 'SpAtt'

bids_path = BIDSPath(
    subject=subject, session=session, task=task, run="1", datatype="meg", root=output_path
)
write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events=events_fname,
    event_id=event_dict,
    empty_room=raw_er,
    overwrite=True,
)

#### Print newly created BIDS Directory

In [ ]:
print_dir_tree(output_path)